<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Skriptsprachen
### Sommersemester 2021
Prof. Dr. Heiner Giefers

# Dekoratoren
Mit Dekoratoren kann man die Kernfunktionalität von Funktionen und Klassen um bestimmte Aspekte erweitern. Das Programmierparadigma hinter dieser Idee heißt daher auch [_Aspektorientierte Programmierung_](https://de.wikipedia.org/wiki/Aspektorientierte_Programmierung).

Dekoratoren sind Funktionen, die andere Funktionen "abändern" bzw. erweitern können. Wie normale Funktionen können auch Dekoratoren über einen Funktionsaufruf benutzt werden. Angenommen, `f()` ist eine Funktion und `mydecorator()` unsere Dekoratorfunktion. Wir können nun `f()` "dekorieren", indem wir
```python
f = mydecorator(f)
```
aufrufen. Das gleiche erreicht man, indem man der Funktionsdefinition ein `@mydecorator` voranstellt. Also z.B.:
```python
@mydecorator
def f():
    pass
```

## Funktionsdekoratoren
Bevor wir uns anschauen, wie man Funktionsdkoratoren entwickelt, sollten wir uns nochmals einige Aspekte von Funktionen in Python vor Augen führen.

*1.* Funktionsnamen sind Referenzen auf Funktionen, daher kann ein und dieselbe Funktion über mehrere Namen aufgerufen werden.

In [ ]:
def original():
    print("Hallo vom Original!")

original()
kopie=original
kopie()

Ein Funktionsname ist nur ein Zeiger auf ein ausführbares Objekt - die eigentliche Funktion - und kann ggf. auch gelöscht werden. Das eigentliche Funktionsobjekt bleibt erhalten, solange noch eine Referenz auf das Objekt existiert.

In [ ]:
del original
kopie()

*2.* Funktionen können innerhalb von Funktionen definiert werden.

In [ ]:
def f():
    def g():
        print("Hallo aus g()", end='')
    g()

f()

*3.* Funktionen können Referenzen auf Funktionen als Resultat zurückgeben.

In [ ]:
def f():
    def g():
        print("Hallo aus g()", end='')
    return g

x=f()
x()

*4.* Funktionsreferenzen können als Argument an andere Funktionen übergeben werden.

In [ ]:
def g():
    print("Hallo aus g()", end='')
def h():
    print("Hallo aus h()", end='')   
def f(x):
    x()
    print(" in f()")
          
f(g)
f(h)

Mittels dieser vier Eigenschaften lassen sich nun Funktionsdekoratoren wie im folgenden Beispiel definieren.

## Logging-Dekorator
Ein Anwendungsfall für Dekoratoren ist z.B. die Erzeugung von Debug-Informationen (_Logging_). Angenommen man möchte, dass bestimmte Funktionen zur Laufzeit eine Ausgabe generieren, wenn sie Betreten und Verlassen werden. Dazu müssten normalerweise die entsprechenden Funktionen durch `print`-Anweisungen am Anfang und Ende des Funktionskörpers versehen werden. Solche Änderungen sind aufwendig zu implementieren, da sie in jeder Funktion gleichartig hinzugefügt und, nach dem Testen, wieder entfernt werden müssen.

Viel eleganter ist es, die Logging-Funktionalität zentral zu definieren und auf die entsprechenden Funktionen zu übertragen; die folgende Funktion `mylogger` ist ein Beispiel dafür. 

In [ ]:
def mylogger(func):
    def foo(*args, **kwargs):
        print("Betrete %s" %  func.__name__)
        func(*args, **kwargs)
        print("Verlasse %s" % func.__name__)
    return foo

`mylogger` bekommt als Argument eine Referenz auf ein Funktionsobjekt `func`. Der Funktionskörper von `mylogger` besteht hauptsächlich aus der Definition einer neuen Funktion `foo`. Diese Funktion führt die Funktion `func` mit ihrem eigenen Parameter-Tupel `(*args, **kwargs)` aus und erzeugt vor und nach dem Aufruf eine Ausgabe.
Die generelle Formulierung der Parameter mit `(*args, **kwargs)` erlaubt es, die Funktion `foo` mit einer beliebigen, gültigen Liste von Parametern aufzurufen.
Am Ende gibt `mylogger` eine Referenz auf die Funktion `foo` als Resultat zurück.
Wenn man also `mylogger` folgendermaßen auf eine Funktion `f` anwendet, definiert man die ursprüngliche Funktion um, und `f` erhält eine veränderte Funktionalität:

In [ ]:
def f():
    pass
    
f=mylogger(f)
f()

Ein etwas umfangreicheres Beispiel mit mehreren Funktionen:

In [ ]:
def g():
    print("Hallo aus g()", end='')
def h():
    print("Hallo aus h()", end='')  
def f(x):
    x()
    print(" in f()")
    
g=mylogger(g)
f=mylogger(f)

f(g)
f(h)

In [ ]:
@mylogger
def g():
    print("Hallo aus g()", end='')
def h():
    print("Hallo aus h()", end='')
@mylogger    
def f(x):
    x()
    print(" in f()")

f(g)
f(h)

**Aufgabe 1**

**Entwickeln Sie einen Funktionsdekorator mit dem Sie die Laufzeit von Funktionsaufrufen auf der Standardausgabe ausgeben können.**

Hinweise: Aus einer früheren Aufgabe kennen Sie bereits das Modul _time_. Wir hatten die Methoden _strftime()_ und _gmtime()_ folgendermaßen benutzt, um einen Zeitstempel zu erzeugen:
```python
import time
time.strftime("%d.%m.%Y %H:%M:%S", time.gmtime())
```
Für Laufzeitmessungen bietet sich eher die Methode _time()_ aus dem _time_-Modul an. Sie liefert ein _float_-Wert zurück, der den vergangenen Sekunden seit beginn der [UNIX Zeit](https://de.wikipedia.org/wiki/Unixzeit) entspricht.

In [ ]:
import time
help(time.time)

Um einen Code-Abschnitt zu messen, können sie _time()_ z.B. so einsetzen:

In [ ]:
import time
zeiten={'millisec':{'faktor':1e-3, 'einheit':'ms'},
        'microsec':{'faktor':1e-6, 'einheit':'mu'}}
ordn='millisec'
t0 = time.time()
print("Die Laufzeit für die Ausgabe dieses Textes wird gemessen.")
t1 = time.time()
print("Start: %f Ende: %f Dauer: %f%s" % (t0, t1, (t1-t0)/zeiten[ordn]['faktor'], zeiten[ordn]['einheit']))

In [ ]:
def function_timer(func):
    '''
    This decorator prints execution time in ms.
    '''
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test Cell
#----------
from io import StringIO
from mock import patch
#----------
# testing function
@function_timer
def test(a, b, *p):
    time.sleep(0.1)
    return a+b
#----------
# callability
assert callable(test), 'Decorator should return a callable object.'
#----------
# argument flexibility
try: test(1,2,3,4,5,6)
except TypeError: raise AssertionError('The decorator should pass the arguments of the')
#----------
assert test('Test ', 'successful!') == 'Test successful!', 'The result of the function should be returned'

with patch('sys.stdout', new_callable=StringIO) as screen:
    test(1,1)
    assert '10' in screen.getvalue(), 'Timer should print approx 100 ms'
#----------

**Aufgabe 2**

**Testen Sie ihren Dekorator Funktion**

Hinweise: Zum Testen Ihres Dekorators ist eine Funktion mit relativ langer Laufzeit hilfreich. In _Übungsblatt 1_ haben Sie eine rekursive Variante der Fibonacci-Funktion kennengelernt und bereits das Laufzeitverhalten analysiert. Verwenden Sie die folgende funktion `fib(n)` mit Ihrem Dekorator. Was fällt Ihnen auf? Wie kann man Problem umgehen?

In [ ]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)
    
fib_timed = None #implement your timer here
# YOUR CODE HERE
raise NotImplementedError()
fib_timed(20)

In [ ]:
# Test Cell
#----------

assert callable(fib_timed)
assert fib_timed.__code__.co_nlocals > 1

## Caching
Ein weiteres Anwendungsgebiet für das sich Dekoratoren gut eignen ist _Caching_. Caches sind für den Benutzer transparente (also "unsichtbare") Puffer Speicher, die den Zugriff auf Informationen beschleunigen. In der Informatik werden viele verschiedene Formen von Cache-Speichern angewendet. Nahezu jeder Prozessor besitzt einen oder mehrere Cache Speicher; aber auch in Software sind Caching-Methoden allgegenwärtig. Ein Cache ist immer vor eine Informationsquelle geschaltet. Also in Hardware z.B. vor den Hauptspeicher, in Software z.B. als sogenannter look-up Speicher vor einen Funktionsaufruf.

Der folgende Dekorator `cached` implementiert einen Cache für Funktionen mit einem Parameter. Wird eine "dekorierte" Funktion ausgeführt, so wird zunächst im Dictionary _cache_ nachgeschaut, ob ein Eintrag für den aktuellen Wert des Parameters `x` existiert. Ist das der Fall, so wird der Wert an der Stelle `x` im Dictionary _cache_ ausgelesen und als Resultat zurück gegeben. Ist der Wert des Parameters `x` nicht im cache (d.h., die Funktion wurde zuvor noch nicht mit dem Argument `x` aufgerufen) wird zuerst die Funktion `f(x)` berechnet und ihr Ergebnis in `cache[x]` eingetragen.

In [ ]:
def cached(f):
    cache = {}
    global debug
    if debug: print("Erzeuge cache Objekt @%s" % id(cache))
    def foo(x): 
        if x not in cache:            
            cache[x] = f(x)
            if debug: print('Packe "%s" in den cache %s' % (x, id(cache)))
        else:
            if debug: print('"%s" ist im cache %s' % (x, id(cache)))
        return cache[x]
    return foo

Der Dekorator wird jeweils nur einmal, zu dem Zeitpunkt, wenn eine Funktion "dekoriert" wird angewendet. Daher existiert auch das Dictionary _cache_ nur einmal pro Funktionsobjekt. Im folgenden Beispiel dekorieren wir die Funktionen _m1_ und _m2_ mit _cached_. Somit wird zweimal ein Dictionary _cache_ angelegt. Die Dictionaries sind aber nicht lokal innerhalb der Funktionen _m1_ und _m2_ definiert. In dem Fall würde bei jedem Funktionsaufruf ein neues Dictionary Objekt angelegt (was die Funktion des Caches zerstören würde). Vielmehr verweisen die Funktionen  _m1_ und _m2_ nur auf die beiden _cache_-Objekte, die bei den beiden Aufrufen der _cached_-Funktion erzeugt wurden.

In [ ]:
debug=True

@cached
def m1(s):
    return s
@cached
def m2(s):
    return s
    
print(m1("Hello"))
print(m2("Hello"))
print(m1("Hello"))
print(m2("Hello"))

# Zusätzliche Ausgaben abschalten:
debug=False

Nun können wir die Fibonacci-Funktion mit dem Cache benutzen.

In [ ]:
@cached
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

**Aufgabe 3**

**Vergleichen Sie das Laufzeitverhalten der Fibonacci-Funktion mit und ohne Caching**

In [ ]:
@cached
def fib(n):
    return n if n <=1 else (fib(n-1) + fib(n-2))

fib.__name__

In [ ]:
@cached
def fib(n):
    return n if n <=1 else (fib(n-1) + fib(n-2))

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test Cell
#----------

assert fib.__name__ == 'fib', 'fib function should be defined twice, with and without the cached!'